<a href="https://colab.research.google.com/github/markoutsikou/DWS101-ML/blob/main/ML_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplot.pyplot as plt
import plotly.express as px
import yfinance as yf
from datetime import datetime
import holidays


ModuleNotFoundError: No module named 'matplot'